<a href="https://colab.research.google.com/github/SerArtDev/redes-neuronales/blob/main/redes_neuronales_introducci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción a redes neuronales artificiales

Se quiere predecir la nota que obtendrá un estudiante basado en sus horas de estudio y de sueño.

| Horas de sueño | Horas de estudio | Nota |
|----------------|------------------|------|
| 3              | 5                | 75   |
| 5              | 1                | 82   |
| 10             | 2                | 93   |


In [ ]:
import numpy as np

x = np.array(([3, 5], [5, 1], [10, 2]), dtype=float)
y = np.array(([75], [82], [93]), dtype=float)

Normalizamos los datos, de tal manera que estén en un rango entre 0 y 1. Además, con eso aseguramos tener datos independientes de unidades entre las variables predictoras y la de respuesta.

$x_{norm} = \frac{x}{x_{max}}$

$y_{norm} = \frac{y}{y_{max}}$

Para este caso, sabemos que la nota máxima es 100, entonces ese puede ser $y_{max}$.

In [ ]:
x_norm = x / np.amax(x, axis=0)
y_norm = y / 100

Tenemos una capa de dos neuoras de entrada y una capa con una neurona de salida. Entre estas dos capas se encuentran las llamadas capas ocultas, que si son muchas con muchas neuronas se le llamaría deep learning. Para este caso, utilizaremos tres neuronas en la capa oculta.

Entre la primera y segunda capa se tienen 6 conexiones (2x3) y entre la segunda y tercera capa hay 3 conexiones (3x1). Cada una de estas conexiones tiene un peso $w$. En la segunda capa, para cada neurona se calcula la activación $z$ como la suma entre los valores $x$ de cada neurona de la capa anterior por el peso de la sinapsis o conexión entre estas dos.

Los pesos en la sinapsis se pueden representar de la forma:

$$
w^I =
\begin{bmatrix}
w_{1,1} & w_{1,2} & w_{1,3} \\
w_{2,1} & w_{2,2} & w_{2,3} \\
\end{bmatrix}
$$

El primer subíndice de cada elemento es la neurona de la primer cada y el segundo es a la neura que esta está conectada, entonces, $w_{1,1}$ representa al peso de la conexión entre la primer neurona de la primer cada con la primera neurona de la segunda capa.

Se puede representar $z$ de forma matricial.

 $ z^{II} = x*w^I$.


A partir de $z$ se puede calcular la activación usando una función de activación $f$.

$a^{II} = f(z^{II})$

Para la siguiente capa, ya no se usa $x$, sino $a^{II}$.

$z^{III}=a^{II}*w^{II}$

Estas conexiones tienen los pesos $w^{II}$.

$$
w^{II} =
\begin{bmatrix}
w_{1,1} \\
w_{2,1} \\
w_{3,1} \\
\end{bmatrix}
$$

A este $z^{III}$ se le aplica la función de activación, obteniendo así el valor de salida de la red neuronal.

$\hat{y} = f(z^{III})$

In [ ]:
class Neural_network(object):
  def __init__(self):
    self.input_layer_size = 2
    self.output_layer_size = 1
    self.hidden_layer_size = 3
    # Pesos de las sinapsis
    self.wi = np.random.randn(self.input_layer_size, self.hidden_layer_size)
    self.wii = np.random.randn(self.hidden_layer_size, self.output_layer_size)

  ## Se calcula el valor de salida de la red.
  def forward(self, x):
    self.zii = np.dot(x, self.wi)
    self.aii = self.sigmoid(self.zii)
    self.ziii = np.dot(self.aii, self.wii)
    y_hat = self.sigmoid(self.ziii)
    return y_hat


  ## Función de activación Sigmoid
  def sigmoid(self, z):
    return 1 / (1 + np.exp(-z))


  ## ----------------------Optimización------------------------------

  def sigmoid_prime(self, z):
    ## Derivdad de la función de activación Sigmoid
    return np.exp(-z) / ((1 + np.exp(-z))**2)


  def cost_funtion(self, x, y):
    self.y_hat = self.forward(x)
    return 0.5 * sum((y - self.y_hat)**2)


  def cost_function_prime(self, x, y):
    # Cálculo de derivada respecto a wi y wii
    self.y_hat = self.forward(x)
    deltaiii = np.multiply(-(y-self.y_hat), self.sigmoid_prime(self.ziii))
    dJdwii = np.dot(self.aii.T, deltaiii)

    deltaii = np.dot(deltaiii, self.wii.T) * self.sigmoid_prime(self.zii)
    dJdwi = np.dot(x.T, deltaii)

    return dJdwi, dJdwii

  ## --------------Descenso de gradiente numérico------------------
  def get_params(self):
    params = np.concatenate((self.wi.ravel(), self.wii.ravel()))
    return params

  def set_params(self, params):
    wi_start = 0
    wi_end = self.hidden_layer_size * self.input_layer_size
    self.wi = np.reshape(params[wi_start:wi_end],
                        (self.input_layer_size, self.hidden_layer_size)
                        )

    wii_end = wi_end + self.hidden_layer_size*self.output_layer_size
    self.wii = np.reshape(params[wi_end:wii_end],
                        (self.hidden_layer_size, self.output_layer_size)
                        )

  def compute_gradients(self, x, y):
    dJwi, dJdwii = self.cost_function_prime(x, y)
    return np.concatenate(dJwi.ravel(), dJdwii.ravel())


  def compute_numerical_gradient(self, x, y):
    params_initial = self.get_params()
    numgrad = np.zeros(params_initial.shape)
    perturb = np.zeros(params_initial.shape)
    e = 1e-4

    for p in range(len(params_initial)):
      # Vector de perturbación
      perturb[p] = e
      self.set_params(params_initial + perturb)
      loss2 = self.cost_funtion(x, y)
      self.set_params(params_initial - perturb)
      loss1 = self.cost_funtion(x, y)
      # Calcular el gradiente
      numgrad[p] = (loss2 - loss1) / (2*e)
      perturb[p] = 0

    self.set_params = params_initial

    return numgrad

  def optimize(self, x, y, alpha):
    while self.cost_funtion(x,y) > 10e-2:
      print(self.cost_funtion(x,y))
      self.set_params(self.get_params() + alpha * self.compute_numerical_gradient(x, y))


In [ ]:
nn = Neural_network()
y_hat = nn.forward(x)
print(y_hat)

[[0.28535098]
 [0.32615166]
 [0.31820229]]


## Optimización
Los valores obtenidos dependen de los pesos de las sinapsis y estos deben ser ajustados optimizando el modelo, de forma que pueda predecir valores de salida con mayor exactitud. Para esto, comparamos los valores de saluda reales ($y$) con los valores calculados con la red ($\hat{y}$) para sus valores correspondientes de entrada ($x$).

$ J = \sum \frac{1}{2}(y-\hat{y})^2 $

J es la función de costo y este debe ser minimizada ajustando $w^{I}, w^{II}$.

$\hat{y} = f(f(x*w^I)*w^{II})$

$J = \sum \frac{1}{2}(y-f(f(x*w^I)*w^{II}))^2$

El método de descenso de gradiente es el comúnmente usado para optimizar(entrenar) redes neuronales. Para este es necesario calcular las derivadas parciales de la función de costo respecto a $w^I, w^{II}$. Esto se logra aplicando la regla de la cadena múltiples veces y, en parte, por esto se llama backpropagation.

$\frac{\delta J}{\delta w^{II}}=(a^{II})^T\delta^{III}$

$\delta^{III}=-(y-\hat{y})f'(z^{III})$

$\frac{\delta J}{\delta w^I}=x^T\delta^{II}$

$\delta^{II}=\delta^{III}(W^{II})^Tf'(z^{II})$

In [ ]:
nn = Neural_network()
cost1 = nn.cost_funtion(x, y)
#dJdwi, dJdwii = nn.cost_function_prime(x, y)

print(cost1)

[10272.58582891]


A partir de estos gratientes se pueden obtener valores óptimos para las $w$.

$w^I_1=w^I_0+\alpha \frac{\delta J}{\delta w^I_0}$

$w^{II}_1=w^{II}_0+\alpha \frac{\delta J}{\delta w^{II}_0}$

In [ ]:
nn.optimize(x, y, 1)


[10272.58582891]
[10499.]


<ipython-input-3-53c69c9c3bcf>:83: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  numgrad[p] = (loss2 - loss1) / (2*e)


TypeError: 'numpy.ndarray' object is not callable